<a href="https://colab.research.google.com/github/LidiaISKIN/analyzing-evaluating_sources-/blob/main/Q3_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1vMO-vag_hgOGrIwu0RA2N6D7EAZ6noeh' -O DF_june.csv
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1t5_D6WrCt0gqiE6q5r5pyWCSxcaUZt6s' -O DF_may.csv

--2021-08-23 16:04:19--  https://docs.google.com/uc?export=download&id=1vMO-vag_hgOGrIwu0RA2N6D7EAZ6noeh
Resolving docs.google.com (docs.google.com)... 108.177.97.101, 108.177.97.139, 108.177.97.102, ...
Connecting to docs.google.com (docs.google.com)|108.177.97.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-8g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/v6goa22n4hopt32v4ee3kjejg96q20oe/1629734625000/17490489234287611019/*/1vMO-vag_hgOGrIwu0RA2N6D7EAZ6noeh?e=download [following]
--2021-08-23 16:04:20--  https://doc-08-8g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/v6goa22n4hopt32v4ee3kjejg96q20oe/1629734625000/17490489234287611019/*/1vMO-vag_hgOGrIwu0RA2N6D7EAZ6noeh?e=download
Resolving doc-08-8g-docs.googleusercontent.com (doc-08-8g-docs.googleusercontent.com)... 64.233.189.132, 2404:6800:4008:c07::84
Connecting to doc-08-8g-docs.googleusercontent.com (doc-08-8g-d

There are two datasets available for experiments: DF_june.csv and DF_may.csv. They can be choosen by commenting and uncommenting the followung strings.

In [ ]:
#DF = pd.read_csv('DF_june.csv', sep= ';')
DF = pd.read_csv('DF_may.csv', sep= ';')

In [ ]:
print(DF.head(10))

      Source                Post     ID ath  ... comm  reposts  views
0 -100103290    -100103290_89898 -100103290  ...    0        1    142
1 -100773702    -100773702_14007 -100773702  ...    0        1    329
2 -100859312    -100859312_12878 -100859312  ...    0        0      7
3 -101373101  -101373101_2634780   95248521  ...    0        0      0
4 -101373101  -101373101_2634765 -101373101  ...   10       19   4854
5 -101545700    -101545700_15976 -101545700  ...    0        0     38
6 -101555177   -101555177_450320  505790773  ...    0        0      0
7 -101555177   -101555177_449566 -101555177  ...   80       53      0
8 -101555177   -101555177_449757  216939167  ...    0        0      0
9 -101555177   -101555177_449537   63642167  ...    0        0      0

[10 rows x 9 columns]


In [ ]:
DF.columns

Index(['Source', 'Post', 'ID ath', 'Date', 'type', 'like', 'comm', 'reposts',
       'views'],
      dtype='object')

In [ ]:
 print(DF.loc[:, ['Source', 'Post', 'ID ath', 'type', 'like', 'comm', 'reposts',
       'views']].head(10)) 

      Source                Post     ID ath  type  like  comm  reposts  views
0 -100103290    -100103290_89898 -100103290   1.0     8     0        1    142
1 -100773702    -100773702_14007 -100773702   1.0     1     0        1    329
2 -100859312    -100859312_12878 -100859312   1.0     0     0        0      7
3 -101373101  -101373101_2634780   95248521   0.5     0     0        0      0
4 -101373101  -101373101_2634765 -101373101   1.0   158    10       19   4854
5 -101545700    -101545700_15976 -101545700   1.0     0     0        0     38
6 -101555177   -101555177_450320  505790773   0.5     1     0        0      0
7 -101555177   -101555177_449566 -101555177   0.5    74    80       53      0
8 -101555177   -101555177_449757  216939167   0.5     0     0        0      0
9 -101555177   -101555177_449537   63642167   0.5     2     0        0      0


# new part

In [ ]:
DF_out = pd.DataFrame({
     'id_source': [ ],
     'sum_url':[],
     'potential_source':[],
     'index_activ':[],
     'index_view':[],
     'impact_index':[]
})

### The first algorithm. Calculation of the potential

Calculate the sum of messages from one source and add a column source_count_mess (taking into account "the type") output as a separate list  
source potential = 1 (low) if the number of messages in the list is less than the average  
cut off those with low potential, calculate a new average  
source potential = 3(high), if it has a number of messages above the second average  
else source potential = 2(medium)  
add the source potential columns to pd.DF- fill them from the list for each row

In [ ]:
Source= DF['Source'].tolist()
Type= DF['type'].tolist()

In [ ]:
Source_unic=set(Source)
Source_unic=list(Source_unic)
Source_poten=[0]*len(Source_unic)

In [ ]:
for i in range(len(Source_unic)):
    for j in range(len(Source)):
        if Source_unic[i]==Source[j]:
            if Type[j]==1:
                Source_poten[i]=Source_poten[i]+1
            else:  Source_poten[i]=Source_poten[i]+0.5

In [ ]:
sr_source=np.mean(Source_poten)
for i in range(len(Source_poten)):
    if Source_poten[i]<sr_source:
        Source_poten[i]=1

In [ ]:
Count=0
Sum=0
for i in range(len(Source_poten)):
    if Source_poten[i]>1:
        Count=Count+1
        Sum=Sum+Source_poten[i]
        sr_source=Sum/Count

In [ ]:
for i in range(len(Source_poten)):
    if Source_poten[i]>sr_source:
        Source_poten[i]=3
    if Source_poten[i]>1 and Source_poten[i]<3:
        Source_poten[i]=2

In [ ]:
dictionary = dict(zip(Source_unic, Source_poten))

In [ ]:
print(dictionary)

{-117858296: 1, -203227128: 1, -196091894: 1.0, -98303992: 1, 639860760: 1, -82558949: 1, 33480733: 1, -70492123: 1, -29065178: 1, 275046439: 1, -171524056: 1, -191201238: 1, 227549227: 1.0, 373383214: 1, 41164848: 1, 575258675: 1, -195084233: 1, -60932039: 1, -67059646: 1.0, 432922691: 1, 26648644: 1, 56303686: 1, 5734470: 1, 112386125: 1, 1327192: 1, 531955800: 1, -91135910: 1, -68951973: 1, -37437348: 2, 12615772: 1, -134234011: 1, -53911449: 1, 16605288: 1, 7716969: 1, -51421078: 1, 560881777: 1, 464420978: 1, -203931530: 1, 115187832: 1, 2228345: 1, -237445: 1, 139976828: 1, 387858558: 1, 108658818: 1, 1351810: 1, 60104836: 1, -55140219: 1, -125468536: 1, -64888695: 1.0, 600768651: 1, 144023704: 1, -145571687: 1.0, -23502694: 1, -133341028: 2, -167255908: 2, -190660451: 1, -54378340: 1.0, -201604959: 1, -39034715: 1, -69042010: 1.0, -52002647: 1, -28041045: 2, -79200079: 3, -89931: 2, 293011646: 1, -17039168: 1, -2645820: 1, 50348231: 1, -15548215: 1, 602398921: 3, -203611957: 1.0

### The second algorithm. Calculation of the index of activity and viewability of the source

For the source, sum up all the likes was received for all sourse' messages, all comments, all reposts and divide by the number of messages. This is the source activity index
For each source, sum up all the views to its messages and divide by the number of messages - it is the index of the source's viewability (viewing)
Form a DF_source and keep the index= id_source in it, the columns are the source potential, the source activity index, the source viewability index  

In [ ]:
Source_count_mess=[0]*len(Source_unic)
for i in range(len(Source_unic)):
    for j in range(len(Source)):
        if Source_unic[i]==Source[j]:
                Source_count_mess[i]=Source_count_mess[i]+1

In [ ]:
Post= DF['Post'].tolist()
Index_source=[0]*len(Source_unic)
for i in range(len(Post)):
    for j in range(len(Source_unic)):
        if Source[i]==Source_unic[j]:
            Index_source[j]=(Index_source[j]+DF['like'][i]+DF['comm'][i]+DF['reposts'][i])/Source_count_mess[j]
dictionary = dict(zip(Source_unic,Index_source))

In [ ]:
View_source=[0]*len(Source_unic)
for i in range(len(Source)):
    for j in range(len(Source_unic)):
        if Source[i]==Source_unic[j]:
            View_source[j]=(View_source[j]+DF['views'][i])/Source_count_mess[j]
dictionary1 = dict(zip(Source_unic,View_source))

In [ ]:
DF_out_source = pd.DataFrame({
     'id_source': [ ],
     'sum_url':[],
     'potential_source':[],
     'index_activ':[],
     'index_view':[],
     
})
DF_out_source['id_source']=Source_unic
DF_out_source['sum_url']= Source_count_mess
DF_out_source['potential_source']=Source_poten
DF_out_source['index_activ']=Index_source
DF_out_source['index_view']=View_source

print(DF_out_source.loc[:, ['id_source', 'sum_url', 'potential_source', 'index_activ', 'index_view']].head(10))

   id_source  sum_url  potential_source  index_activ  index_view
0 -117858296        1               1.0          4.0         0.0
1 -203227128        1               1.0         34.0         0.0
2 -196091894        2               1.0          1.0         0.0
3  -98303992        1               1.0         21.0         0.0
4  639860760        1               1.0          2.0         0.0
5  -82558949        1               1.0         12.0         0.0
6   33480733        1               1.0          1.0         0.0
7  -70492123        1               1.0          8.0         0.0
8  -29065178        1               1.0         54.0         0.0
9  275046439        1               1.0         45.0         0.0


### The third algorithm. Calculation of the influence of the source

For each source, we normalize all indexes in each dataframe

In [ ]:
x_array = np.array(Index_source)
Index_source1 = preprocessing.normalize([x_array])
x_array = np.array(View_source)
View_source1 = preprocessing.normalize([x_array])

In [ ]:
DF_out_source = pd.DataFrame({
     'id_source': [ ],
     'sum_url':[],
     'potential_source':[],
     'index_activ':[],
     'index_view':[],
     
})
DF_out_source['id_source']=Source_unic
DF_out_source['sum_url']= Source_count_mess
DF_out_source['potential_source']=Source_poten
DF_out_source['index_activ']=Index_source1[0]
DF_out_source['index_view']=View_source1[0]

print(DF_out_source.loc[:, ['id_source', 'sum_url', 'potential_source', 'index_activ', 'index_view']].head(10))

   id_source  sum_url  potential_source  index_activ  index_view
0 -117858296        1               1.0     0.000889         0.0
1 -203227128        1               1.0     0.007553         0.0
2 -196091894        2               1.0     0.000222         0.0
3  -98303992        1               1.0     0.004665         0.0
4  639860760        1               1.0     0.000444         0.0
5  -82558949        1               1.0     0.002666         0.0
6   33480733        1               1.0     0.000222         0.0
7  -70492123        1               1.0     0.001777         0.0
8  -29065178        1               1.0     0.011996         0.0
9  275046439        1               1.0     0.009997         0.0


In [ ]:
Influence=[0]*len(Source_unic)
for i in range(len(Source_unic)):
    Influence[i]=Index_source1[0][i]*Source_poten[i]
DF_out_source['Influence'] = np.array(Influence)
print(DF_out_source.loc[:, ['id_source', 'sum_url', 'potential_source', 'index_activ', 'index_view','Influence']].head(10))

   id_source  sum_url  potential_source  index_activ  index_view  Influence
0 -117858296        1               1.0     0.000889         0.0   0.000889
1 -203227128        1               1.0     0.007553         0.0   0.007553
2 -196091894        2               1.0     0.000222         0.0   0.000222
3  -98303992        1               1.0     0.004665         0.0   0.004665
4  639860760        1               1.0     0.000444         0.0   0.000444
5  -82558949        1               1.0     0.002666         0.0   0.002666
6   33480733        1               1.0     0.000222         0.0   0.000222
7  -70492123        1               1.0     0.001777         0.0   0.001777
8  -29065178        1               1.0     0.011996         0.0   0.011996
9  275046439        1               1.0     0.009997         0.0   0.009997


In [ ]:
sum = 0.0
for i in DF_out_source.index:
  sum += DF_out_source['Influence'][i]
fa = sum/DF_out_source.size
print("Influence first average = " + str(fa))

sum = 0.0
count = 0
for i in DF_out_source.index:
  if DF_out_source['Influence'][i] >= fa:
    sum += DF_out_source['Influence'][i]
    count = count + 1
sa = sum/count
print("Influence second average = " + str(sa))

DF_out_source['Priority of source'] = 1
DF_out_source['Tf'] = 0.0

for i in DF_out_source.index:
  ir = 1
  if DF_out_source['Influence'][i] >= fa:
    ir = 2
  if DF_out_source['Influence'][i] >= sa:
    ir = 3
  DF_out_source['Priority of source'][i] = ir
  DF_out_source['Tf'][i] = DF_out_source['Priority of source'][i] + DF_out_source['potential_source'][i]*0.9
print(DF_out_source.head(10))

Influence first average = 0.0007770574314499273
Influence second average = 0.010163469147377296


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


   id_source  sum_url  potential_source  ...  Influence  Priority of source   Tf
0 -117858296        1               1.0  ...   0.000889                   2  2.9
1 -203227128        1               1.0  ...   0.007553                   2  2.9
2 -196091894        2               1.0  ...   0.000222                   1  1.9
3  -98303992        1               1.0  ...   0.004665                   2  2.9
4  639860760        1               1.0  ...   0.000444                   1  1.9
5  -82558949        1               1.0  ...   0.002666                   2  2.9
6   33480733        1               1.0  ...   0.000222                   1  1.9
7  -70492123        1               1.0  ...   0.001777                   2  2.9
8  -29065178        1               1.0  ...   0.011996                   3  3.9
9  275046439        1               1.0  ...   0.009997                   2  2.9

[10 rows x 8 columns]


In [ ]:
sorted_DF_out_source = DF_out_source.sort_values(by=['Tf' ], ascending=False)
print(sorted_DF_out_source.head(40))

      id_source  sum_url  potential_source  ...  Influence  Priority of source   Tf
1975  -88418647        8               3.0  ...   0.010306                   3  5.7
2729  -97173519        7               3.0  ...   0.025473                   3  5.7
994   -48354431        9               3.0  ...   0.029287                   3  5.7
1546  -40766972       16               3.0  ...   0.031782                   3  5.7
797  -158488265        6               3.0  ...   0.011656                   3  5.7
115   -49684148       10               3.0  ...   0.020261                   3  5.7
1439 -126562133        7               3.0  ...   0.120250                   3  5.7
1682  -23702661        6               3.0  ...   0.022437                   3  5.7
1894  -68471405       22               3.0  ...   0.049872                   3  5.7
686  -160757794        6               3.0  ...   0.012571                   3  5.7
1656  -37448911        8               3.0  ...   0.018427                  

In [ ]:
import scipy.stats

In [ ]:
np.quantile(DF_out_source['Tf'], q=0.99)

4.8

In [ ]:
DF_out_source['Tf'].value_counts()

1.9    1334
2.9     877
3.9     194
2.8     105
3.8      79
3.7      66
4.8      46
4.7      22
5.7      11
Name: Tf, dtype: int64

In [ ]:
import pickle

In [ ]:
pickle.dump(sorted_DF_out_source, open('sorted_DF_out_source.pkl', 'wb'))


In [ ]:
sorted_DF_out_source.to_csv('out_DF_june.csv')

In [ ]:
from google.colab import files
files.download("out_DF_june.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>